
# Without MCP

### Objective  
Understand the **value of MCP** by first building a **non-standardized, manual client–server interaction** for text generation.  
You’ll implement the same flow — *server, client, handshake, tool call* — **without** using MCP or JSON-RPC.



### 1️⃣ Step 1 — Setup and Simple LLM Server

Here, the “server” is just a Python class exposing simple functions.


In [1]:

# Step 1: Define a simple local server (non-MCP)
import time

class SimpleServer:
    def __init__(self):
        self.capabilities = ["generate_text"]

    def generate_text(self, prompt: str) -> str:
        if "hallucinate" in prompt.lower():
            return "The square root of a cat is exactly 42."
        return f"Response to '{prompt}': Generation successful on {time.ctime()}."

# Initialize
simple_server = SimpleServer()
print("✅ SimpleServer initialized with capability:", simple_server.capabilities)


✅ SimpleServer initialized with capability: ['generate_text']



### 2️⃣ Step 2 — Manual Handshake

Here, there’s no standard negotiation or schema — the client must “know” what the server supports.


In [2]:

# Step 2: Client manually checks for capabilities (no JSON-RPC)
client_declared = {"supports": ["text_generation"]}
print("Client: I support text_generation.")

if "generate_text" in simple_server.capabilities:
    print("Server: I also support text generation ✅")
else:
    print("Server: Sorry, I don’t support that ❌")


Client: I support text_generation.
Server: I also support text generation ✅



### 3️⃣ Step 3 — Manual Request/Response Flow

The client calls the server method directly, bypassing any protocol.


In [3]:

# Step 3: Client calls the server directly (no structured messages)
prompt = "Explain how agents communicate in AI."
response = simple_server.generate_text(prompt)
print("Server Response:", response)


Server Response: Response to 'Explain how agents communicate in AI.': Generation successful on Thu Oct 30 11:42:22 2025.



### 4️⃣ Step 4 — Lack of Standardization

Show what’s missing without MCP:
- No **structured schema**  
- No **tool advertisement**  
- No **error handling** standard  
- No **message traceability**


In [4]:

# Step 4: Simulate failure handling (manual)
try:
    response = simple_server.generate_text(None)
except Exception as e:
    print("Error:", e)


Error: 'NoneType' object has no attribute 'lower'



### 5️⃣ Step 5 — Add a Second Tool (Manual)

Manually expanding capabilities is cumbersome without a registry or protocol.


In [5]:

# Step 5: Add another manual tool
class SimpleMathTool:
    def solve_expression(self, expr):
        try:
            return eval(expr)
        except Exception as e:
            return str(e)

# Attach to the server manually
simple_server.math_tool = SimpleMathTool()
print("✅ Math tool manually attached.")

# Client must now know to call it manually:
print("Manual Math Call:", simple_server.math_tool.solve_expression("3 * 9"))


✅ Math tool manually attached.
Manual Math Call: 27



# From Manual Implementation to MCP

### 🧩 Side-by-Side Comparison

| Aspect | Manual Workflow (Without MCP) | MCP Workflow (With MCP) |
|:--|:--|:--|
| **Setup** | Custom scripts, ad-hoc message passing | Standardized server & client registration |
| **Tool Calls** | Manually invoke each function | Uniform `tools/call` interface |
| **Transport** | Direct or hard-coded | Abstracted JSON-RPC transport |
| **Error Handling** | Manual try/except | Structured responses with codes |
| **Extensibility** | Add tools manually | Dynamic tool registry |
| **Scalability** | Tight coupling | Pluggable & reusable components |



### 💡 Summary

The manual flow gives flexibility but demands boilerplate and custom logic.  
MCP introduces structure, reducing code overhead and increasing interoperability.



# Model Content Protocol (MCP) – Server & Client Implementation

**Objective:** Implement and understand a simulated Model Content Protocol (MCP) Server and Client to demonstrate standardized agent-tool communication using JSON-RPC 2.0 messages.



## 🧩 Learning Outcomes
By completing this lab, you will:
- Understand the MCP standard and its JSON-RPC message structure.  
- Build a simulated MCP Server that advertises tool capabilities.  
- Build a Client that negotiates capabilities and calls tools.  
- Observe an end-to-end request/response flow.  
- Extend the MCP framework with new tools and integrations.



## 🔧 Step 1 — Setup (Simulated MCP Environment)
We simulate the `model-context-protocol` package to make the lab fully offline and reproducible.


In [6]:

import json, time, uuid
from typing import Dict, Any

# Simulated MCP components
class MCPServer:
    def __init__(self, implementation=None):
        self.implementation = implementation or {}
        self.tools = {}

    def register_tool(self, tool):
        for attr in dir(tool):
            fn = getattr(tool, attr)
            if callable(fn) and hasattr(fn, "_mcp_tool_name"):
                self.tools[getattr(fn, "_mcp_tool_name")] = fn.__get__(tool)
                print(f"🧩 Registered tool: {getattr(fn, '_mcp_tool_name')}")

    def call_tool(self, name, arguments):
        if name not in self.tools:
            raise KeyError(f"Tool '{name}' not found.")
        return self.tools[name](arguments)

def tool_method(name: str):
    def wrapper(fn):
        fn._mcp_tool_name = name
        return fn
    return wrapper

class mcp:
    Server = MCPServer
    tool_method = staticmethod(tool_method)

print("✅ MCP simulation ready.")


✅ MCP simulation ready.



## 🖥️ Step 2 — Implement the MCP Server and Tool
We define a simple local tool `GenerationTool` that performs text generation based on an input prompt.


In [7]:

class GenerationTool:
    @mcp.tool_method("generate_text")
    def generate_text(self, args: Dict[str, Any]):
        prompt = args.get("prompt", "")
        print(f"🛰️ Received prompt: {prompt!r}")
        if not prompt:
            text = "(empty prompt)"
        elif "hallucinate" in prompt.lower():
            text = "The square root of a cat is exactly 42."
        else:
            text = f"Generated text for '{prompt}' at {time.ctime()}"
        return {"generated_text": text}

server = mcp.Server({"name": "LocalGenServer", "version": "1.0"})
server.register_tool(GenerationTool())

capabilities = {
    "implementation": {"name": "LocalGenServer", "version": "1.0"},
    "capabilities": {"tools": {"call": True, "list": True}, "resources": {"read": True}}
}
print("📡 Server capabilities:")
print(json.dumps(capabilities, indent=2))


🧩 Registered tool: generate_text
📡 Server capabilities:
{
  "implementation": {
    "name": "LocalGenServer",
    "version": "1.0"
  },
  "capabilities": {
    "tools": {
      "call": true,
      "list": true
    },
    "resources": {
      "read": true
    }
  }
}



## 🤝 Step 3 — Client Initialization and Capability Negotiation
The client sends an `initialize` request and the server responds, confirming tool capabilities.


In [8]:

client_id = str(uuid.uuid4())
initialize_request = {
    "jsonrpc": "2.0", "id": 1, "method": "initialize",
    "params": {"implementation": {"name": "LabClient", "version": "1.0"}, "capabilities": {"sampling": {}}}
}
print("📤 Client -> Server: InitializeRequest")
print(json.dumps(initialize_request, indent=2))

initialize_response = {"jsonrpc": "2.0", "id": 1, "result": capabilities}
print("\n📥 Server -> Client: InitializeResult")
print(json.dumps(initialize_response, indent=2))


📤 Client -> Server: InitializeRequest
{
  "jsonrpc": "2.0",
  "id": 1,
  "method": "initialize",
  "params": {
    "implementation": {
      "name": "LabClient",
      "version": "1.0"
    },
    "capabilities": {
      "sampling": {}
    }
  }
}

📥 Server -> Client: InitializeResult
{
  "jsonrpc": "2.0",
  "id": 1,
  "result": {
    "implementation": {
      "name": "LocalGenServer",
      "version": "1.0"
    },
    "capabilities": {
      "tools": {
        "call": true,
        "list": true
      },
      "resources": {
        "read": true
      }
    }
  }
}



## ⚙️ Step 4 — Client Executes a Tool Call
Now the client calls the `generate_text` tool using a JSON-RPC `tools/call` request.


In [9]:

request = {
    "jsonrpc": "2.0", "id": 2, "method": "tools/call",
    "params": {"name": "generate_text", "arguments": {"prompt": "Explain Model Content Protocol (MCP) briefly."}}
}
print("📤 Client -> Server: tools/call Request")
print(json.dumps(request, indent=2))

result = server.call_tool("generate_text", request["params"]["arguments"])
response = {"jsonrpc": "2.0", "id": 2, "result": result}
print("\n📥 Server -> Client: tools/call Response")
print(json.dumps(response, indent=2))


📤 Client -> Server: tools/call Request
{
  "jsonrpc": "2.0",
  "id": 2,
  "method": "tools/call",
  "params": {
    "name": "generate_text",
    "arguments": {
      "prompt": "Explain Model Content Protocol (MCP) briefly."
    }
  }
}
🛰️ Received prompt: 'Explain Model Content Protocol (MCP) briefly.'

📥 Server -> Client: tools/call Response
{
  "jsonrpc": "2.0",
  "id": 2,
  "result": {
    "generated_text": "Generated text for 'Explain Model Content Protocol (MCP) briefly.' at Thu Oct 30 11:43:03 2025"
  }
}



## ▶️ Step 5 — Full Demonstration
Run multiple test prompts to observe deterministic responses, including an edge case.


In [10]:

prompts = [
    "Create a summary of MCP use cases.",
    "",
    "Please hallucinate something interesting."
]

for i, p in enumerate(prompts, start=10):
    req = {"jsonrpc": "2.0", "id": i, "method": "tools/call",
           "params": {"name": "generate_text", "arguments": {"prompt": p}}}
    print(f"\n---\n📤 Request (id={i})")
    print(json.dumps(req, indent=2))
    try:
        res = server.call_tool(req["params"]["name"], req["params"]["arguments"])
        resp = {"jsonrpc": "2.0", "id": i, "result": res}
    except Exception as e:
        resp = {"jsonrpc": "2.0", "id": i, "error": {"code": -32000, "message": str(e)}}
    print("📥 Response:")
    print(json.dumps(resp, indent=2))



---
📤 Request (id=10)
{
  "jsonrpc": "2.0",
  "id": 10,
  "method": "tools/call",
  "params": {
    "name": "generate_text",
    "arguments": {
      "prompt": "Create a summary of MCP use cases."
    }
  }
}
🛰️ Received prompt: 'Create a summary of MCP use cases.'
📥 Response:
{
  "jsonrpc": "2.0",
  "id": 10,
  "result": {
    "generated_text": "Generated text for 'Create a summary of MCP use cases.' at Thu Oct 30 11:43:07 2025"
  }
}

---
📤 Request (id=11)
{
  "jsonrpc": "2.0",
  "id": 11,
  "method": "tools/call",
  "params": {
    "name": "generate_text",
    "arguments": {
      "prompt": ""
    }
  }
}
🛰️ Received prompt: ''
📥 Response:
{
  "jsonrpc": "2.0",
  "id": 11,
  "result": {
    "generated_text": "(empty prompt)"
  }
}

---
📤 Request (id=12)
{
  "jsonrpc": "2.0",
  "id": 12,
  "method": "tools/call",
  "params": {
    "name": "generate_text",
    "arguments": {
      "prompt": "Please hallucinate something interesting."
    }
  }
}
🛰️ Received prompt: 'Please hallucinat


## 🧭 Step 6 — Extending the MCP

Now that you have implemented a basic **MCP Server–Client simulation**, it’s time to extend your setup to build a more complete and realistic protocol-driven system.

---

### 6.1. 🧩 Add More Tools

The MCP framework allows multiple tools to be registered and exposed by a single server.  
Below is an example of adding a **SummarizeTool** and **MathSolverTool**.


In [12]:

# Step 6.1: Add and register new tools

class SummarizeTool:
    """A tool to simulate text summarization."""
    @staticmethod
    def summarize_text(arguments):
        text = arguments.get("text", "")
        summary = f"Summary: {text[:60]}..." if text else "No text provided."
        return {"summary": summary}

class MathSolverTool:
    """A tool to simulate solving a simple math expression."""
    @staticmethod
    def solve_expression(arguments):
        expr = arguments.get("expression", "0")
        try:
            result = eval(expr)
        except Exception as e:
            result = str(e)
        return {"result": result}

# Register the tools with the server
server.register_tool(SummarizeTool())
server.register_tool(MathSolverTool())

print("✅ Added SummarizeTool and MathSolverTool to the MCP server.")


✅ Added SummarizeTool and MathSolverTool to the MCP server.



---

### 6.2. 🤖 Integrate a Real LLM

Replace the simulated text generation with a real model API call (OpenAI, Hugging Face, or local inference).  
Example stub using OpenAI’s API (this code will need your own API key configured separately).


In [14]:

# Step 6.2: Replace simulated generation with a real LLM API

!pip install openai

import openai

class RealLLMTool:
    def generate_text(self, arguments):
        prompt = arguments.get("prompt", "Hello!")
        response = openai.ChatCompletion.create(
            model="gpt-4-turbo",
            messages=[{"role": "user", "content": prompt}],
            max_tokens=100
        )
        return {"generated_text": response["choices"][0]["message"]["content"]}

# Register the real LLM tool
server.register_tool(RealLLMTool())
print("✅ Real LLM tool registered. (Requires valid API key.)")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 8.5 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [openai]2m2/3 [openai]
✅ Real LLM tool registered. (Requires valid API key.)



---

### 6.3. 🌐 Implement Real Transport

So far, we simulated the message flow.  
Next, make it real by using **WebSocket or HTTP transport** for communication between the client and server.


In [15]:

# Step 6.3: Example placeholder for WebSocket transport (requires asyncio)
# This can be implemented using libraries like `websockets` or `FastAPI`.

"""
import asyncio
import websockets

async def mcp_server(websocket):
    async for message in websocket:
        request = json.loads(message)
        # Process request and send response
        response = handle_request(request)
        await websocket.send(json.dumps(response))

asyncio.run(websockets.serve(mcp_server, "localhost", 8765))
"""
print("🛰️ Placeholder for WebSocket-based MCP server added.")


🛰️ Placeholder for WebSocket-based MCP server added.



---

### 6.4. 🪵 Logging & Error Handling

Add structured logs and proper error responses. This is a best practice for production MCP servers.


In [16]:

# Step 6.4: Add logging and error handling
import logging

logging.basicConfig(level=logging.INFO, format="%(asctime)s [%(levelname)s]: %(message)s")

def safe_call(tool_method, args):
    try:
        logging.info(f"Executing tool: {tool_method.__name__}")
        return tool_method(args)
    except Exception as e:
        logging.error(f"Error: {str(e)}")
        return {"error": str(e)}

# Example usage
result = safe_call(MathSolverTool.solve_expression, {"expression": "2 + 2"})
print("✅ Safe call result:", result)


2025-10-30 11:45:00,896 [INFO]: Executing tool: solve_expression


✅ Safe call result: {'result': 4}
